# Evaluating Machine Learning Models

## Simple Hold-out Validation

In [ ]:
# Hold-out Validation

num_validation_samples = 10000
np.random.shuffle(data)
validation_data = data[:num_validation_samples] # validation set
data = data[num_validation_samples:] # training set

training_data = data[:]

model = get_model()
model.train(training_data)
validation_score = model.evaluate(validation_data)

# At this point you can tune your model
# retrain it, evaluate it, tune it again

model = get_model()
model.train(np.concatenate([training_data,
                            validation_data]))
test_score = model.evaluate(test_data)

Veriseti az ise bu problem oluşturacaktır ve farklı random shuffling değerleri çok farklı sonuçlar veriyorsa muhtemelen az verimiz var demektir.

# K-Fold Validation

In [ ]:
# K-fold cross-validation

k = 4
num_validation_samples = len(data) // k

np.random.shuffle(data)

validation_scores = []
for fold in range(k):
  validation_data = data[num_validation_samples * fold: num_validation_samples * (fold + 1)]
  training_data = data[:num_validation_samples * fold] + data[num_validation_samples * (fold + 1):]

  model = get_model()
  model.train(training_data)
  validation_score = model.evaluate(validation_data)
  validation_scores.append(validation_score)

validation_score = np.average(validation_scores)

model = get_model()
model.train(data)
test_score = model.evaluate(test_data)

**Value normalization**

Normalize each feature independently to have a mean of 0.

Normaliza each feature independently to have a standard deviation of 1.


In [ ]:
# Value normalization

# Assuming x is a 2D data matrix of shape
x -= x.mean(axis=0)
x /= x.std(axis=0)

## Reducing the network's size

In [ ]:
# Modelin tüm parametrelerini beslemek için büyük networkler için büyük verisetleri gerekir bu yüzden de veriye bağlı küçük networkle başlamak ya da
# modeli küçültmek overfitting gibi durumlara yakalanmamamız için önemlidir.

# Original model
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(16, activation="relu", input_shape=(10000,)))
model.add(layers.Dense(16, activation="relu"))
model.add(layers.Dense(1, activation="sigmoid"))

# Version of the model with lower capacity
model = models.Sequential()
model.add(layers.Dense(4, activation="relu", input_shape=(10000,)))
model.add(layers.Dense(4, activation="relu"))
model.add(layers.Dense(1, activation="sigmoid"))

In [ ]:
# Version of the model with higher capacity
model = models.Sequential()
model.add(layers.Dense(512, activation="relu", input_shape=(10000,)))
model.add(layers.Dense(512, activation="relu"))
model.add(layers.Dense(1, activation="sigmoid"))

# Simpler models are less likely to overfit than complex ones.

In [ ]:
# To mitigate overfitting, use L1, L2 regularization, L2 regularization = weight decay

# Adding L2 weight regularization to the model

from keras import regularizers

model = models.Sequential()
model.add(layers.Dense(16, kernel_regularizer=regularizers.l2(0.001),
                       activation="relu", input_shape=(10000,)))
model.add(layers.Dense(16, kernel_regularizer=regularizers.l2(0.001),
                       activation="relu"))
model.add(layers.Dense(16, activation="sigmoid"))

# l2(0.001) means every coefficient in the weight matrix of the layer will add 0.001 * weight_coefficient_value

# As an alternative to L2 regularization

regularizers.l1(0.001) # L1 regularization

regularizers.l1_l2(l1=0.001, l2=0.001) # L1 and L2 regularization

In [ ]:
# Adding dropout
# Dropout is one of the most effective and most commonly used regularization techniques for neural networks

layer_output *= np.random.randint(0, high=2, size=layer_output.shape) # At training time, drops out %50 of the units in the output
# At test time, we scale down the output by the dropout rate
layer_output *= 0.5 # At test time

# Adding dropout to the IMDB network
model = models.Sequential()
model.add(layers.Dense(16, activation="relu", input_shape=(10000,)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(16, activation="relu"))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation="sigmoid"))